In [44]:
import pandas as pd
import re
import numpy as np
import google.generativeai as genai
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import csv

Nossa base está com um problema, com alguns comentários sendo repartidos em mais de 1 coluna. Ao fazermos a leitura do nosso csv, vamos corrigir para que essas partes estejam todas unidas em 1 único comentário.

In [45]:
# Lista para armazenar as linhas corrigidas
corrected_rows = []

# Abrir o arquivo CSV para leitura e correção
with open('base de dados.csv', 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile, delimiter=',', quotechar='"')
    for row in reader:
        # Verifica o número de colunas e une as colunas conforme necessário
        num_columns = len(row)
        if num_columns >= 21:
            # Une as colunas de acordo com o número total de colunas
            end_col = min(num_columns - 1, 15)  # Até a coluna 15 no máximo
            row[10] = ' '.join(row[10:end_col])  # Une as colunas 11 a 15
            # Remove as colunas extras
            del row[11:end_col]
        corrected_rows.append(row)

# Cria um DataFrame a partir das linhas corrigidas
df_reviews = pd.DataFrame(corrected_rows[1:], columns=corrected_rows[0])  # Assume que a primeira linha é o cabeçalho

# Exibe as primeiras linhas do DataFrame
df_reviews.head()

,id,appId,createdAt,date,detectedLang,lang,replyDate,replyText,score,store,text,title,url,userImage,userName,version,category,isGeneric,sentiment,subcategory
0,39f3958d-3ace-42e4-8feb-b799e9f254e7,com.meuposto,2024-08-20 13:08:21,2024-08-19 13:00:12,pt,pt,NULL,NULL,1,google,quando vcs vao arrumar o aplicativo? Manter ta...,NULL,https://play.google.com/store/apps/details?id=...,https://play-lh.googleusercontent.com/a-/ALV-U...,Leonardo A Gerbelli,4.62.110,Complaint,0,Negative,Performance
1,37a0c658-ef11-49b8-a784-b27f42e7ea39,com.meuposto,2024-08-16 04:04:40,2024-08-19 12:42:05,NULL,pt,2024-08-19 12:39:08,"Rodrigo, para garantir a segurança de nossos s...",4,google,Ok,NULL,https://play.google.com/store/apps/details?id=...,https://play-lh.googleusercontent.com/a-/ALV-U...,Rodrigo Prodossimo,4.62.110,Compliment,1,Positive,Generic compliment
2,8db5accc-380e-41aa-be0c-ac6ea6e38a8d,com.meuposto,2024-08-20 12:08:45,2024-08-19 11:57:28,pt,pt,NULL,NULL,1,google,"Faz uma semana que estou com app, até agora nã...",NULL,https://play.google.com/store/apps/details?id=...,https://play-lh.googleusercontent.com/a/ACg8oc...,Igor Machado,4.62.110,Complaint,0,Negative,Login
3,086b8680-c871-4f14-9fa3-0ebe55a0c965,com.meuposto,2024-08-15 12:07:40,2024-08-19 11:53:24,pt,pt,2024-08-16 18:40:39,"Olá, Jeff! Saiba que esse não é o nosso padrão...",1,google,"CONTINUA NÃO FUNCIONANDO, O SUPORTE INFORMADO ...",NULL,https://play.google.com/store/apps/details?id=...,https://play-lh.googleusercontent.com/a/ACg8oc...,Jeff Jr.,4.62.110,Complaint,0,Negative,Performance
4,d1fc787b-127c-47e4-8324-9885247336c6,com.meuposto,2024-08-20 12:08:45,2024-08-19 11:43:27,NULL,pt,2024-08-20 13:04:11,"Olá, Jair! Que ótimo receber sua avaliação pos...",5,google,Muito bom.,NULL,https://play.google.com/store/apps/details?id=...,https://play-lh.googleusercontent.com/a/ACg8oc...,Jair Cabral,4.62.110,Compliment,1,Positive,Generic compliment


Vamos remover algumas colunas que não vamos utilizar. A princípio, sabemos que todos os dados são do mesmo cliente, então vamos dropar "appId". Vamos dropar as colunas sobre língua, as colunas sobre reply e as com links

In [46]:
# Especificar as colunas que você deseja remover
df_reviews_drop = ['appId', 'createdAt', 'detectedLang', 'lang', 'replyDate', 'replyText', 'url', 'userImage']

# Dropar as colunas
df_reviews = df_reviews.drop(columns=df_reviews_drop)

df_reviews.head()

,id,date,score,store,text,title,userName,version,category,isGeneric,sentiment,subcategory
0,39f3958d-3ace-42e4-8feb-b799e9f254e7,2024-08-19 13:00:12,1,google,quando vcs vao arrumar o aplicativo? Manter ta...,NULL,Leonardo A Gerbelli,4.62.110,Complaint,0,Negative,Performance
1,37a0c658-ef11-49b8-a784-b27f42e7ea39,2024-08-19 12:42:05,4,google,Ok,NULL,Rodrigo Prodossimo,4.62.110,Compliment,1,Positive,Generic compliment
2,8db5accc-380e-41aa-be0c-ac6ea6e38a8d,2024-08-19 11:57:28,1,google,"Faz uma semana que estou com app, até agora nã...",NULL,Igor Machado,4.62.110,Complaint,0,Negative,Login
3,086b8680-c871-4f14-9fa3-0ebe55a0c965,2024-08-19 11:53:24,1,google,"CONTINUA NÃO FUNCIONANDO, O SUPORTE INFORMADO ...",NULL,Jeff Jr.,4.62.110,Complaint,0,Negative,Performance
4,d1fc787b-127c-47e4-8324-9885247336c6,2024-08-19 11:43:27,5,google,Muito bom.,NULL,Jair Cabral,4.62.110,Compliment,1,Positive,Generic compliment


Vamos também separar a data do horário, para que possamos manipular mais facilmente esses dados depois.

In [47]:
# Separar a coluna "date" em "date_only" e "time"
df_reviews[['date_only', 'time']] = df_reviews['date'].str.split(' ', expand=True)
df_reviews = df_reviews.drop(columns='date')

#Voltando com o nome "date" para a nova coluna "date_only"
new_name = {'date_only': 'date'}
df_reviews = df_reviews.rename(columns=new_name)

df_reviews

,id,score,store,text,title,userName,version,category,isGeneric,sentiment,subcategory,date,time
0,39f3958d-3ace-42e4-8feb-b799e9f254e7,1,google,quando vcs vao arrumar o aplicativo? Manter ta...,NULL,Leonardo A Gerbelli,4.62.110,Complaint,0,Negative,Performance,2024-08-19,13:00:12
1,37a0c658-ef11-49b8-a784-b27f42e7ea39,4,google,Ok,NULL,Rodrigo Prodossimo,4.62.110,Compliment,1,Positive,Generic compliment,2024-08-19,12:42:05
2,8db5accc-380e-41aa-be0c-ac6ea6e38a8d,1,google,"Faz uma semana que estou com app, até agora nã...",NULL,Igor Machado,4.62.110,Complaint,0,Negative,Login,2024-08-19,11:57:28
3,086b8680-c871-4f14-9fa3-0ebe55a0c965,1,google,"CONTINUA NÃO FUNCIONANDO, O SUPORTE INFORMADO ...",NULL,Jeff Jr.,4.62.110,Complaint,0,Negative,Performance,2024-08-19,11:53:24
4,d1fc787b-127c-47e4-8324-9885247336c6,5,google,Muito bom.,NULL,Jair Cabral,4.62.110,Compliment,1,Positive,Generic compliment,2024-08-19,11:43:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,c4651419-6018-43dc-90a4-6f6f9daf8745,5,google,Otimoooo,NULL,pablo lopes de melo,4.61.109,Complaint,1,Positive,Generic complaint,2024-07-01,11:27:27
1615,e51362fa-9faa-47d9-88de-c3beeac4a85e,1,google,Instabilidade no sistema não deixa o app funci...,NULL,Fe Lu,4.60.107,Complaint,0,Negative,Performance,2024-07-01,10:46:22
1616,acb430bd-c477-42ca-bf07-1454c1313b5c,5,google,App parou de funcionar na hora de realizar pag...,NULL,Alessandro Melim Quelhas,4.61.109,Complaint,0,Negative,Performance,2024-07-01,09:15:01
1617,db24f96e-5180-42d6-98e4-0bafb19f39e2,5,google,Muito bom,NULL,Valdiran santos Pressão,4.61.109,Compliment,1,Positive,Generic compliment,2024-07-01,07:51:44


In [48]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1619 entries, 0 to 1618
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1619 non-null   object
 1   score        1619 non-null   object
 2   store        1619 non-null   object
 3   text         1619 non-null   object
 4   title        1619 non-null   object
 5   userName     1619 non-null   object
 6   version      1619 non-null   object
 7   category     1619 non-null   object
 8   isGeneric    1609 non-null   object
 9   sentiment    1608 non-null   object
 10  subcategory  1608 non-null   object
 11  date         1619 non-null   object
 12  time         1619 non-null   object
dtypes: object(13)
memory usage: 164.6+ KB


Todas nossas colunas numéricas estão com dtype "object". Vamos alterar "date" para datetime, já que isso será útil posteriormente para a análise. Nossa coluna "score" que representa as notas atribuídas aos comentários também deveria ser float.

In [49]:
# Converter o dtype das colunas:
df_reviews['date'] = pd.to_datetime(df_reviews['date'])

Vamos dropar também as linhas com elementos nulos, uma vez que não nos interessa linhas que não tem "sentiment" ou "subcategory" atribuídas. Além disso, elas são poucas linhas em relação ao nosso total e não vão afetar muito os resultados da nossa pesquisa.

In [50]:
#Dropando linhas com elementos nulos nas colunas sentiment e subcategory:
df_reviews = df_reviews.dropna(subset=['sentiment', 'subcategory'])

pd.options.mode.chained_assignment = None
#Transformando score para float, agora que não temos colunas Null para atrapalhar:
df_reviews['score'] = pd.to_numeric(df_reviews['score'])

In [51]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1608 entries, 0 to 1618
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           1608 non-null   object        
 1   score        1608 non-null   int64         
 2   store        1608 non-null   object        
 3   text         1608 non-null   object        
 4   title        1608 non-null   object        
 5   userName     1608 non-null   object        
 6   version      1608 non-null   object        
 7   category     1608 non-null   object        
 8   isGeneric    1608 non-null   object        
 9   sentiment    1608 non-null   object        
 10  subcategory  1608 non-null   object        
 11  date         1608 non-null   datetime64[ns]
 12  time         1608 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 175.9+ KB


In [52]:
df_reviews.describe()

,score,date
count,1608.000000,1608
mean,2.929104,2024-07-29 20:18:48.358209024
min,1.000000,2024-07-01 00:00:00
25%,1.000000,2024-07-16 00:00:00
50%,3.000000,2024-08-01 00:00:00
75%,5.000000,2024-08-13 00:00:00
max,5.000000,2024-08-19 00:00:00
std,1.874719,NaN


Vemos que está tudo certo com o range do nosso "score" (que deveria ir de 1 a 5), e que as datas vão de 01/07 até 19/08. Vamos deixar ambos os meses para podermos comparar dados futuramente.

In [53]:
df_reviews.duplicated(['id']).sum()

114

Observamos que há comentários repetidos observando nosso identificador único. Vamos retirá-los.

In [54]:
df_reviews_cleaned = df_reviews.drop_duplicates(subset=['id'])
df_reviews_cleaned.duplicated(['id']).sum()

0

Agora temos nosso novo df_reviews_cleaned sem duplicações.

In [55]:
print(df_reviews_cleaned['store'].unique())
print(df_reviews_cleaned['isGeneric'].unique())

['google']
['0' '1' 'NULL']


Ao tentarmos entender as variações de "store" e "isGeneric", nos deparamos com "NULL" presente em "isGeneric", mesmo que nosso info() não exibisse valores nulos. Isso nos leva à conclusão de que "NULL" está sendo lida como uma string. Já tínhamos conseguido perceber isso verificiando o head() da coluna "title". Vamos verificar se isso está acontecendo também em outras colunas.

In [56]:
print(df_reviews_cleaned['id'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['store'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['text'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['title'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['userName'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['version'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['category'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['isGeneric'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['sentiment'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['subcategory'].str.contains('NULL', na=False).sum())
print(df_reviews_cleaned['time'].str.contains('NULL', na=False).sum())

0
0
0
1494
0
211
5
11
5
54
0


As colunas "title", "version", "category", "isGeneric", "sentiment" e "subcategory" têm informações "NULL".

Como observamos acima, só temos a store "google" em nosso dataframe. Dessa forma, sabemos que nossos comentários já não teriam "title" (dado que a Play Store não oferece esse campo). Vamos dropar essa coluna.

A quantidade excessiva de "NULL" em version também é esperada, uma vez que essa informação não é compartilhada por todos os usuários. Vamos apenas alterar "NULL" para NaN.

Em relação aos outros elementos, precisamos que os comentários tenham todas as categorias preenchidas, então iremos dropar linhas com conteúdos "NULL", já que há um baixo volume.

In [57]:
# Dropar a coluna title
df_reviews_cleaned = df_reviews_cleaned.drop(columns='title')

# Substituir NULL por NaN
df_reviews_cleaned.replace('NULL', np.nan, inplace=True)

#Dropar linhas com NaN
df_reviews_cleaned = df_reviews_cleaned.dropna(subset=['sentiment', 'subcategory'])

In [58]:
df_reviews_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1440 entries, 0 to 1618
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           1440 non-null   object        
 1   score        1440 non-null   int64         
 2   store        1440 non-null   object        
 3   text         1440 non-null   object        
 4   userName     1440 non-null   object        
 5   version      1240 non-null   object        
 6   category     1440 non-null   object        
 7   isGeneric    1440 non-null   object        
 8   sentiment    1440 non-null   object        
 9   subcategory  1440 non-null   object        
 10  date         1440 non-null   datetime64[ns]
 11  time         1440 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 146.2+ KB


Agora, vamos salvar nosso dataframe limpo.

In [59]:
#df_reviews_cleaned.to_csv('df_reviews_cleaned.csv', index=False)

A partir da nossa tabela, vamos calcular de uma vez a nota média dos comentários de cada dia.

Vamos contar a quantidade de reviews por estrela, agrupadas por dia, e fazer a média ponderada para chegar na nota média diária ("Ratings of Reviews Average") do dia.

In [60]:
# Agrupando nossos reviews por data
weighted_scores_calculation = df_reviews_cleaned.groupby(['date', 'score']).size().unstack(fill_value=0)

# Reordenar as colunas para garantir que estejam na ordem correta
weighted_scores_calculation = weighted_scores_calculation.sort_index(axis=1)

# Calcular a quantidade de comentários
weighted_scores_calculation['Comments'] = (weighted_scores_calculation[1.0] + weighted_scores_calculation[2.0] + 
                            weighted_scores_calculation[3.0] + weighted_scores_calculation[4.0] + 
                            weighted_scores_calculation[5.0])

# Somar todas as notas
weighted_scores_calculation['Ratings Sum'] = (weighted_scores_calculation[1.0] * 1 + weighted_scores_calculation[2.0] * 2 + 
                             weighted_scores_calculation[3.0] * 3 + weighted_scores_calculation[4.0] * 4 + 
                             weighted_scores_calculation[5.0] * 5)

# Multiplicar cada nota pelo seu peso e somar
weighted_scores_calculation['Weighted Scores Calculation'] = (weighted_scores_calculation[1.0] * 1 * 1 + weighted_scores_calculation[2.0] * 2 * 2 + 
                            weighted_scores_calculation[3.0] * 3 * 3 + weighted_scores_calculation[4.0] * 4 * 4 + 
                            weighted_scores_calculation[5.0] * 5 * 5)

# Calcular a média ponderada das notas de acordo com o peso
weighted_scores_calculation['Ratings of Reviews Average'] = weighted_scores_calculation['Weighted Scores Calculation'] / weighted_scores_calculation['Ratings Sum']

# Exibir o resultado
weighted_scores_calculation


score,1,2,3,4,5,Comments,Ratings Sum,Weighted Scores Calculation,Ratings of Reviews Average
date,,,,,,,,,
2024-07-01,4,1,0,0,13,18,71,333,4.690141
2024-07-02,9,0,2,1,10,22,69,293,4.246377
2024-07-03,6,2,0,0,16,24,90,414,4.600000
2024-07-04,3,0,1,1,7,12,45,203,4.511111
2024-07-05,5,2,1,2,10,20,70,304,4.342857
2024-07-06,6,1,0,0,10,17,58,260,4.482759
2024-07-07,8,1,0,1,4,14,34,128,3.764706
2024-07-08,4,0,0,0,11,15,59,279,4.728814
2024-07-09,11,1,1,0,13,26,81,349,4.308642


In [61]:
weighted_scores_calculation.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50 entries, 2024-07-01 to 2024-08-19
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   1                            50 non-null     int64  
 1   2                            50 non-null     int64  
 2   3                            50 non-null     int64  
 3   4                            50 non-null     int64  
 4   5                            50 non-null     int64  
 5   Comments                     50 non-null     int64  
 6   Ratings Sum                  50 non-null     int64  
 7   Weighted Scores Calculation  50 non-null     int64  
 8   Ratings of Reviews Average   50 non-null     float64
dtypes: float64(1), int64(8)
memory usage: 3.9 KB


Agora vamos salvar essa nossa tabela também, mas vamos manter o index, que são nossas datas.

In [63]:
#weighted_scores_calculation.to_csv('weighted_scores_calculation.csv', index=True)